# Setup

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import datetime #for data type changes
import matplotlib as mpl #for visualizations
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import sklearn as sk #for machine learning analysis methods
import random #for random gen
print('All done!')

In [ ]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"
#to allow multiple outputs from one code cell

# Import

Data connected. Now to read it into some data frame variables!

In [ ]:
book_list=pd.read_csv("goodreads_works.csv") #index_col='work_id' added after taking more kaggle courses, then removed because it screws everything up downstream
#reviews=pd.read_csv("/kaggle/input/book-reviews/goodreads_reviews.csv")

**New Error Unlocked**
/tmp/ipykernel_36/3857527195.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews=pd.read_csv("/kaggle/input/book-reviews/goodreads_reviews.csv")

Pulling the data dictionary into a data frame to get an idea of what columns might be throwing mixed types. Also checking the stats of Kaggle's data model pages. That did not shed any light on things...

In [ ]:
FAQ=pd.read_csv("/kaggle/input/book-reviews/goodreads_data_dictionary.csv")
FAQ
#low_memory=False #did not help on this line
reviews=pd.read_csv("/kaggle/input/book-reviews/goodreads_reviews.csv",low_memory=False) #pandas documentation is not an easy read, so this was a natual second step
reviews.work_id.value_counts()

## Data Exploration - Reviews
First let's see which columns had the mixed types...

In [ ]:
dtypes=pd.DataFrame({'review_id': reviews.review_id.dtype,'user_id': reviews.user_id.dtype,'work_id':reviews.work_id.dtype,'started_at':reviews.work_id.dtype,'read_at':reviews.work_id.dtype,'date_added':reviews.work_id.dtype,'rating':reviews.work_id.dtype,'review_text':reviews.work_id.dtype,'n_votes':reviews.work_id.dtype,'n_comments':reviews.work_id.dtype},index=[0])
dtypes
reviews.dtypes

Well, I am going to guess that the three columns being the issue are 'started_at', 'read_at' and 'date_added' because I would think they should be date/time fields, yet they are reporting as int.

***Oh my god! I went back to a different lesson in Pandas and found the simple way to get the types of the columns.***
😫😲 Now I am embarrassed, but *silver lining* I practiced creating data frames and now I will never forget this command exists! I came up here first because I just copied and pasted the code from the lesson (reviews.dtypes) so I didn't know it existed before going and working on the book_list table.

Next, I am going to look at value counts per column and nulls.

In [ ]:
reviews.review_id.value_counts()
reviews.review_id.isnull().sum()
reviews.user_id.value_counts()
reviews.user_id.isnull().sum()
reviews.work_id.value_counts()
reviews.work_id.isnull().sum()
reviews.started_at.value_counts()
reviews.started_at.isnull().sum()
reviews.read_at.value_counts()
reviews.read_at.isnull().sum()
reviews.date_added.value_counts()
reviews.date_added.isnull().sum()
reviews.rating.value_counts()
reviews.rating.isnull().sum()
reviews.review_text.value_counts()
reviews.review_text.isnull().sum()
reviews.n_votes.value_counts()
reviews.n_votes.isnull().sum()
reviews.n_comments.value_counts()
reviews.n_comments.isnull().sum()

* review_id: all values unique, no nulls
* user_id: 18800 unique values, no nulls
* work_id: 13525 unique values, no nulls
* started_at: 3797 unique values, 347495 nulls
* read_at: 5868 unique values, 112122 nulls
* date_added: 3919 unique values, no nulls
* rating: 5 unique values, 37318 nulls
* review_text:1130794 unique values, no nulls
* n_votes: 605 unique values, no nulls
* n_comments: 228 unique values, no nulls

I wonder if the null ratings have reviews. Perhaps I can impute a rating based on the review. There are 37318 though, so if I choose to impute, I'd need to use a function that has a reliable filter, e.g., if review contains '!' & 'good' = 5.0, if review contains 'good' = 4.0, etc.

In [ ]:
reviews.loc[reviews.rating.isnull()==True]

Based on these results, there doesn't seem to be a reliable way to impute an accurate rating. Also, some reviews aren't indicative of the essence of the book. 

Checking some of the quantifiable options...

In [ ]:
round(reviews.describe(),2)
reviews.n_votes.max()
reviews.n_votes.min()
reviews.n_votes.mean()
reviews.n_votes.median()
reviews.rating.mean()
37318/1143887 #percent of ratings with null

n_votes may or may not be useful as it's heavily skewed...may give less weight if used

## Data Exploration - Works

In [ ]:
book_list.head()

Well, I don't think I need to keep image_url for my purposes. I also want to get a count on nulls in isbn and isbn13 because that data should be available, but the work involved if it's alot...
Kaggle didn't throw a data type error for this table, but I want to confirm what the are.

In [ ]:
#book_list.describe()
data_types=pd.DataFrame({'work_id':book_list.work_id.dtype,'isbn':book_list.isbn.dtype,'isbn13':book_list.isbn13.dtype,'orig_pub_yr':book_list.original_publication_year.dtype,'num_pages':book_list.num_pages.dtype,'description':book_list.description.dtype,'genres':book_list.genres.dtype,'num_reviews':book_list.reviews_count.dtype,'num_text_reviews':book_list.text_reviews_count.dtype,'num_5stars':book_list['5_star_ratings'].dtype,'num_4stars':book_list['4_star_ratings'].dtype,'num_3stars':book_list['3_star_ratings'].dtype,'num_2stars':book_list['2_star_ratings'].dtype,'num_1stars':book_list['1_star_ratings'].dtype, 'num_ratings': book_list.ratings_count.dtype,'avg_rating':book_list.avg_rating.dtype,'similar_books':book_list.similar_books.dtype},index=[0])
data_types
#book_list.5_star_ratings #invalid
#book_list['5_star_ratings'] #valid
#book_list.description[1]
book_list.dtypes


That was fun figuring out the correct way to select a column that starts with a digit!
Also, I wish I knew about how you can call dtypes on the data frame before...it's nice not having to horizontally scroll. 
Type recommendations have been recorded and now we move on the the null testing...

I added in this code section to check for duplicates in the key field because "remove duplicates was part of my project" and I wanted to make sure...we know what happens when you assume.

In [ ]:
book_list.work_id.value_counts()
book_list.shape

In [ ]:
book_list.work_id.isnull().sum()
book_list.isbn.isnull().sum()
book_list.isbn13.isnull().sum()
book_list.original_publication_year.isnull().sum()
book_list.num_pages.isnull().sum()
book_list.description.isnull().sum()
book_list.genres.isnull().sum()
book_list.reviews_count.isnull().sum()
book_list.text_reviews_count.isnull().sum()
book_list['5_star_ratings'].isnull().sum()
book_list['4_star_ratings'].isnull().sum()
book_list['3_star_ratings'].isnull().sum()
book_list['2_star_ratings'].isnull().sum()
book_list['1_star_ratings'].isnull().sum()
book_list.ratings_count.isnull().sum()
book_list.avg_rating.isnull().sum()
book_list.similar_books.isnull().sum()

I find it odd that there are so many nulls in isbn and isbn13. ISBNs are to books as SSNs are to natural-born US citizens. Right now it is not worth it to look up all those missing values.
I can find 18 publication years...that could be useful when curating a reading list. Number of pages would likewise be useful, but right now 730 missing values is not worth figuring out how to find.

Looking deeper into the anomalies of num_pages...
* statistics are weird
* low page books aren't unusual, but after checking a few, they are more than likely incorrect

In [ ]:
book_list.num_pages.describe()
book_list.loc[book_list.num_pages.isnull()==True]

## Findings and Action Items
### Reviews Table
* Data type of started_at, read_at and date_added should be changed from int64 to date/time.
* Data type of review_id, user_id and review_text should be changed from object to string.
* Significant amount of nulls in started_at, read_at and rating. Recommend eliminating started_at and read_at due to lack of relevance to book recommendation.
* 3% of rows in rating are null. Propose two options:
    * Impute the average rating value of 3.8
    * Replace with N/A or None (eliminates mathematical operations for analysis)
* Recommend putting less weight on using rating as an input for recommendation or having a null rating handler function

### Book_List Table
* isbn and isbn13 should be same type, can change to int or string
* original_publish_year can be changed from float to date/time
* num_pages should be changed to int
* Description can be changed to string
    * Consider removing the \n coding during transformation
* Recommendations for nulls
    * isbn and isbn13 replace with Missing if a test shows that 1% do exist but aren't listed, replace with None or Unknown if 1% cannot be found
    * Research original publish years and add values
    * descriptions replace with Missing
    * num_pages consider how the data will be used in recommendations...using non-numeric replacement disallows mathematical functions, using 0 will skew mathematical results, mean or median may be appropriate
* Remove image url column 

# Transformation


## Null Handling
Recommendations for nulls
* isbn and isbn13 replace with Missing if a test shows that 1% do exist but aren't listed, replace with None or Unknown if 1% cannot be found
* Research original publish years and add values
* descriptions replace with Missing
* num_pages consider how the data will be used in recommendations...using non-numeric replacement disallows mathematical functions, using 0 will skew mathematical results, mean or median may be appropriate

* Significant amount of nulls in started_at, read_at and rating.
    * Recommend eliminating started_at and read_at due to lack of relevance to book recommendation.
* 3% of rows in rating are null. Propose two options:
    * Impute the average rating value of 3.8
    * Replace with N/A or None (eliminates mathematical operations for analysis)

In [ ]:
reviews.rating.median()
reviews.rating.mean()

In [ ]:
book_list['description']=book_list.description.fillna('Missing')
book_list['num_pages']=book_list.num_pages.fillna(0)
book_list['num_pages']=book_list.num_pages.fillna(0) #after discussing with my wingman
book_list.head()
reviews['rating']=reviews.rating.fillna(3.8)
reviews.rating.isnull().sum()

A-reasearching we go! First up are publication years...

In [ ]:
#book_list.loc[book_list.original_publication_year.isnull()==True]
#book_list.original_publication_year[52]=2016
book_list.loc[52,'original_publication_year']=2016
book_list.loc[52].original_publication_year
book_list.loc[239].original_publication_year #result: nan
book_list.loc[239,'original_publication_year']=2007
book_list.loc[239].original_publication_year #result: 2007 🎉Yay! I'm changing what I want changed!!
book_list.loc[291,'original_publication_year']=2008
book_list.loc[445,'original_publication_year']=2006
book_list.loc[2638,'original_publication_year']=2012
book_list.loc[3105,'original_publication_year']=2012
book_list.loc[3152,'original_publication_year']=2015
book_list.loc[4284,'original_publication_year']=2014
book_list.loc[10630,'original_publication_year']=2016
book_list.loc[11195,'original_publication_year']=2016
book_list.loc[11219,'original_publication_year']=2001
book_list.loc[11237,'original_publication_year']=2016
book_list.loc[11700,'original_publication_year']=2016
book_list.loc[12111,'original_publication_year']=2016
book_list.loc[12118,'original_publication_year']=2013
book_list.loc[12145,'original_publication_year']=2016
book_list.loc[12195,'original_publication_year']=2016
book_list.loc[13235,'original_publication_year']=2016
#book_list.loc[12235]
#book_list.original_publication_year.isnull().sum() #whoops! I had a typo and changed 12235 by mistake to 2016...better check what it should be
#book_list.loc[12235,'original_publication_year']=2011 #crisis averted
#book_list.loc[12235]

Keeping an eye on those last five with missing isbn/isbn13, only one did not have any...it had an ASIN instead
Replace isbn and isbn13 NaN with "Missing"

In [ ]:
book_list['isbn13']=pd.to_numeric(book_list['isbn13'],errors='coerce').fillna(0).astype('int64').astype('string')#guidance from ChatGPT to force isbn13 to behave
book_list.isbn13.head()
book_list[['isbn','isbn13']]=book_list[['isbn','isbn13']].fillna('Missing')
book_list.isbn.head()
book_list['isbn13']=book_list['isbn13'].replace('0','Missing')
book_list.head()

[consulting with ChatGPT about how to update nulls in a column *in situ*](https://chatgpt.com/share/6876aabe-ff4c-8005-b99c-cf045a8c952c)

## Data Types

* Data type of started_at, read_at and date_added should be changed from int64 to date/time.
* Data type of review_id, user_id and review_text should be changed from object to string.
* isbn and isbn13 should be same type, can change to int or string
* original_publish_year can be changed from float to date/time
* num_pages should be changed to int
* Description can be changed to string
    * Consider removing the \n coding during transformation

In [ ]:
reviews.review_id.astype("string")
reviews.review_text.astype("string")

"One peculiarity to keep in mind (and on display very clearly here) is that columns consisting entirely of strings do not get their own type; they are instead given the object type."
🤦🏼‍♀️Read Mel

In [ ]:
#book_list.isbn.astype(int) #ValueError: cannot convert float NaN to integer
#book_list.isbn.astype(float) #ValueError: could not convert string to float: '080075963X' Fun fact-ISBN check digits are base eleven so they include digits 0-9 and 'X'
#book_list.isbn13.astype(float)
book_list.isbn.astype("string")
#for consistency's sake and because there will be no need for math on ISBN13, let's make it a string too
book_list.isbn13.astype("string")

In [ ]:
#book_list.num_pages.astype(int) #IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer
#need to decide what to do about the nulls...for now change them all to 0 because there are no 0 entries > 1 was the min
#book_list.loc[book_list.num_pages.isnull()] before ChatGPT discussion
book_list['num_pages']=book_list.num_pages.astype(int)
book_list['original_publication_year']=book_list.original_publication_year.astype(int)
book_list.dtypes

Datetime experimentation...

In [ ]:
#temp_year=pd.DataFrame({'Year':book_list.original_publication_year.copy()})
#temp_year.Year.astype(str)
#pd.to_datetime('Year',yearfirst=True,format="mixed")

We are throwing a fit because we do not recognize a year on its own to parse as a date. There isn't a good reason except for "but it's time!" to convert it from an int, so I'm crawling out of this rabbit hole.

## Standardization
Inspect what you expect and expect what you inspect

In [ ]:
#reviews.head()
#reviews.loc[reviews.started_at.notnull()]
reviews=reviews.rename(columns={'started_at': 'started','read_at':'finished','date_added':'review_added','review_text': 'review','n_votes': 'num_votes','n_comments':'num_comments'})

In [ ]:
book_list.head()
book_list=book_list.drop('image_url',axis=1)
reviews=reviews.drop(['started','finished'],axis=1)

book_list=book_list.rename(columns={'original_title':'title','reviews_count':'total_reviews','text_reviews_count':'num_written_reviews','5_star_ratings':'5stars','4_star_ratings':'4stars','3_star_ratings':'3stars','2_star_ratings':'2stars','1_star_ratings':'1star','ratings_count':'total_ratings'})
book_list.head()

## Normalization

In [ ]:
#temp=book_list.copy() before I break anything, let's experiment on a copy
book_list['description']=book_list['description'].str.strip()
book_list['title']=book_list['title'].str.strip()
book_list['author']=book_list['author'].str.strip()
book_list['genres']=book_list['genres'].str.strip()
book_list.head()

Using a temp df again to experiment with 'extract' to get the ".0" off the end of isbn13

In [ ]:
#temp=book_list.copy()
#temp['isbn13']=temp.isbn13.replace('Missing',0)
#temp.isbn13.astype('int64') #python is refusing to convert the ISBN13 to a string, it only treats it as a float

#temp.isbn13.str.extract(r"(['.0'])")
#temp.isbn13.str.rsplit('.',expand=True)
#temp['isbn13']=temp['isbn13'].str[:-2]
#temp.loc[2]
#temp['isbn13']=temp['isbn13'].str.rstrip()
#temp.loc[2]
#temp['isbn13'].apply(str).value_counts()
#temp['isbn13'].apply(lambda x:type(x)).value_counts()
#temp.isbn13.dtype
#temp['isbn13'].apply(lambda x:str(x))
#temp['isbn13'].apply(lambda x:f"{x:.0f})
#temp['isbn13']=temp.isbn13.replace(0,'Missing') #does not replace

#print(temp['isbn13'])

**WTF Mate?**
In the code cell above, I was having issues. Looking back, I think I screwed everything up by converting the float directly to string without the int intermediary. The data was carrying a '.0' that it refused to get rid of despite all my efforts. I researched the internet on my own and even created a [separate testing notebook](https://www.kaggle.com/code/melisandefritzsche/testing) to use for testing concepts and troubleshooting. At a loss, I consulted with my AI partner.
[ChatGPT consultation](https://chatgpt.com/share/6879014b-ffe0-8005-9a77-cf04c80b590f)
*Solution*
I implemented the solution up in the [null handling](https://www.kaggle.com/code/melisandefritzsche/bookshelf-challenge/edit/run/250847672#Null-Handling) section.

In [ ]:
book_list=book_list.set_index('work_id')
reviews=reviews.set_index('review_id')

## Splitting and Extracting

separating the genre lists into individual genres will assist in grouping

In [ ]:
genres=book_list.copy()
genres.shape
book_list.head()
#genre_split=genres['genres'].str.split(',',expand=True)
#genre_split
#genre_split=genres['genres'].str.split(',').explode('genres').reset_index                                                
#genre_split

I am going to try to insert the unexpanded list version of the string into the column using replace and then explode it

In [ ]:
genres['genres']=genres['genres'].str.split(',')
genres.head()

In [ ]:
genres_spl=genres.explode('genres')
genres_spl

🤩**I WIN!**

In [ ]:
genres_spl.genres.value_counts()#checking for duplicates
genres_spl['genres']=genres_spl.genres.str.strip() #previously did not have the genres_spl['genres']=...learning curve observation-I need to get used to syntax for Python just doing a thing versus Pythong doing the thing and keeping it done
#genres_spl.reindex()
#genres_spl.head()
genres_spl.groupby('genres').count().sort_values(by='title',ascending=False)

In [ ]:
#genres_spl=genres_spl.set_index('work_id') #after adding the index set to book_list, this is unneccesary

Just in case for filtering (and for coding practice) I am going to split the author field into first and last name as well.

In [ ]:
temp=book_list.copy()
temp1=temp.author.str.rsplit(' ',n=1,expand=True) #I did something last time -I accidentally undid all my previous work- and inserted the new columns into the original list so I am being redundant as a form of caution
temp1.head()
temp.head()
author_spl=temp
author_spl.insert(4,'first_name',temp1[0])
author_spl.insert(5,'last_name',temp1[1])
author_spl

In [ ]:
#author_spl.dtypes
#author_spl.original_publication_year.astype(int)
author_spl=author_spl.rename(columns={'original_publication_year':'year'})

In [ ]:
#author_pl=author_spl.set_index('work_id')

## Merging and Joining

In [ ]:
#book_list=book_list.set_index('work_id')
#reviews=reviews.set_index('work_id')

all_data=book_list.join(reviews)

*Line in the Sand*
I fully admit that this is not the most efficient way to do things, but my lack of experience with Python and non-math-related analysis goals (I'm curating a book list so to my mind that basically means I'm going to be filtering and sorting. As opposed to giving statistical descriptions or predicting stuff.)
Anyway, I am stopping here with five total data frames to use.
1. all_data - everything joined together
2. book_list - just the book information
3. reviews - just the review information
4. genres_spl - table with all genres split out
5. author_spl - table with authors' first and last names split

# Analysis

## Functions

This whole "build a tool" thing has me a bit confused. I am thinking that I will have a Q&A (not NLP) where I obtain information from the user and then spit out their result. Therefore, I will need helper function to take particular actions like filtering. I should probably define the functionality so that my scope is clear.

* Get how many books the user wants on the list per this query.
* "Surprise Me!": random number generator to pick books by their work_id
* Get preferences - fiction/non-fiction, author, genre
* Filters

In [ ]:
def get_num_books():
    print('How many books would you like on your reading list?')
    num_books=input('Enter a number less than 13,524:')
    return num_books
get_num_books()

In [ ]:
def surprise_me(num_books):
    
    reading_list=random.sample(range(0,13525),k=(num_books))
    return reading_list

surprise_me(5)

In [ ]:
books=surprise_me(3)
temp=book_list.sort_values(by='total_reviews',ascending=False)
print('SURPRISE!! Happy reading!')
temp.iloc[books,lambda temp:[2,3,6,7]]

Take *that*, Python! 😤

In [ ]:
book_list.sort_values(by='total_reviews',ascending=False) #This is how I'd sort my temp list if it were cleaned up and everything. Every time I go back and rerun my code I wind up breaking something so...

## Visualization in Python
Exploring what matplotlib has to offer. Also, instead of stumbling about, I took the Kaggle Data Visualization course and learned about seaborn.

In [ ]:
genres_spl.sort_values(by='total_ratings',ascending=False)

In [ ]:
sns.scatterplot(x='avg_rating',y='total_reviews',data=genres_spl,hue='genres')
plt.title('Average Rating by Genre')
plt.xlabel('Avg Rating')
plt.ylabel('Total Reviews')
plt.legend(loc='upper right',bbox_to_anchor=(1.35,1))
#researched the interwebs and copied but edited
def M_formatter(x, pos):
    """The function formats the tick labels to display in millions."""
    return f'{x / 1_000_000:.1f}M'
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(M_formatter))

Huh. Looks like romance and crime/thriller get reviewed the most and get reviewed highly on average. Young adult and fiction are highest rated even with less overall reviews. That tracks.

In [ ]:
sns.scatterplot(x='avg_rating',y='ratings_count',data=genres_spl,hue='genres')
plt.title('Average Rating by Genre')
plt.xlabel('Avg Rating')
plt.ylabel('Total Reviews')
plt.legend(loc='upper right',bbox_to_anchor=(1.35,1))
#researched the interwebs and copied but edited
def M_formatter(x, pos):
    """The function formats the tick labels to display in millions."""
    return f'{x / 1_000_000:.1f}M'
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(M_formatter))

In [ ]:
sns.barplot(x=book_list.avg_rating,y=book_list.num_pages)


In [ ]:
sns.histplot(x=book_list.avg_rating,y=book_list.num_pages)